<a href="https://colab.research.google.com/github/raissabispo/dashboard-IOT/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly --quiet

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [ ]:
CHANNEL_ID = "2943258"
URL = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?results=50"

In [ ]:
resp = requests.get(URL)
resp.raise_for_status()
data = resp.json()

#DataFrame
feeds = data.get("feeds", [])
df = pd.DataFrame(feeds)
df = df[["created_at", "field1", "field2"]].rename(
    columns={"created_at": "DataHora", "field1": "Umidade", "field2": "Temperatura"}
)

#Conversões
df["DataHora"] = pd.to_datetime(df["DataHora"], utc=True)
df["DataHora"] = df["DataHora"].dt.tz_convert("America/Sao_Paulo")
df["Umidade"] = pd.to_numeric(df["Umidade"], errors="coerce")
df["Temperatura"] = pd.to_numeric(df["Temperatura"], errors="coerce")

# Traces
trace_umid = go.Scatter(
    x=df["DataHora"], y=df["Umidade"],
    mode="lines+markers",
    name="Umidade (%)",
    line=dict(shape="spline", width=2.5, color="blue"),
    marker=dict(size=6, color="blue"),
    hovertemplate="%{y:.1f}%<br>%{x|%H:%M, %d/%m}"
)

trace_temp = go.Scatter(
    x=df["DataHora"], y=df["Temperatura"],
    mode="lines+markers",
    name="Temperatura (°C)",
    line=dict(shape="spline", width=2.5, color="red"),
    marker=dict(size=6, color="red"),
    hovertemplate="%{y:.1f}°C<br>%{x|%H:%M, %d/%m}"
)

#Dashboard com botões
fig = go.Figure(data=[trace_umid, trace_temp])

fig.update_layout(
    title="Dashboard IoT - Umidade & Temperatura",
    template="plotly_white",
    plot_bgcolor="rgba(250,250,250,1)",
    paper_bgcolor="rgba(250,250,250,1)",
    xaxis_title="Data/Hora",
    yaxis_title="Valor",
    xaxis=dict(showgrid=True, gridcolor="#e6e6e6"),
    yaxis=dict(showgrid=True, gridcolor="#e6e6e6"),
    legend=dict(
        orientation="h",
        yanchor="bottom", y=1.02,
        xanchor="center", x=0.5
    ),
    margin=dict(l=40, r=20, t=60, b=60),
    updatemenus=[dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.1, y=1.15,
        buttons=[
            dict(label="Ambos",
                 method="update",
                 args=[{"visible": [True, True]},
                       {"title": "Umidade & Temperatura"}]),
            dict(label="Umidade",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": "Umidade"}]),
            dict(label="Temperatura",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Temperatura"}]),
        ]
    )]
)

# Últimas leituras
fig.add_annotation(
    x=df["DataHora"].iloc[-1],
    y=df["Umidade"].iloc[-1],
    text=f"{df['Umidade'].iloc[-1]:.1f}%",
    showarrow=False,
    font=dict(color="blue", size=12),
    bgcolor="rgba(255,255,255,0.7)",
    borderpad=3
)

fig.add_annotation(
    x=df["DataHora"].iloc[-1],
    y=df["Temperatura"].iloc[-1],
    text=f"{df['Temperatura'].iloc[-1]:.1f}°C",
    showarrow=False,
    font=dict(color="red", size=12),
    bgcolor="rgba(255,255,255,0.7)",
    borderpad=3
)

fig.show()

# Relatório resumido
relatorio = pd.DataFrame({
    "Métrica": ["Umidade (%)", "Temperatura (°C)"],
    "Média": [df["Umidade"].mean(), df["Temperatura"].mean()],
    "Mínimo": [df["Umidade"].min(), df["Temperatura"].min()],
    "Máximo": [df["Umidade"].max(), df["Temperatura"].max()],
    "Última Leitura": [df["Umidade"].iloc[-1], df["Temperatura"].iloc[-1]]
})

